# DataFrames: Reading in messy data
     
In the [01-data-access](./01-data-access.ipynb) example we show how Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  One key difference, when using Dask Dataframes is that instead of opening a single file with a function like [pandas.read_csv](https://docs.dask.org/en/latest/generated/dask.dataframe.read_csv.html), we typically open many files at once with [dask.dataframe.read_csv](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.read_csv). This enables us to treat a collection of files as a single dataset. Most of the time this works really well. But real data is messy and in this notebook we will explore a more advanced technique to bring messy datasets into a dask dataframe.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=True, memory_limit='2GB')
client

<Client: 'tcp://127.0.0.1:36405' processes=1 threads=4, memory=1.86 GiB>

## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [2]:
import dask
df = dask.datasets.timeseries()
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
2000-01-01      int64  object  float64  float64
2000-01-02        ...     ...      ...      ...
...               ...     ...      ...      ...
2000-01-30        ...     ...      ...      ...
2000-01-31        ...     ...      ...      ...
Dask Name: make-timeseries, 30 tasks

In [3]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name, index=False);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [4]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [5]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
                int64  object  float64  float64
                  ...     ...      ...      ...
...               ...     ...      ...      ...
                  ...     ...      ...      ...
                  ...     ...      ...      ...
Dask Name: read-csv, 30 tasks

In [6]:
df.head()

id      name         x         y
0  1054     Wendy -0.858609  0.475144
1   955   Michael  0.938475  0.654481
2   996     Quinn -0.681506 -0.339557
3  1039     Zelda  0.052902  0.689524
4  1009  Patricia -0.313526 -0.959217

Let's look at some statistics on the data

In [7]:
df.describe().compute()

id             x             y
count  2.592000e+06  2.592000e+06  2.592000e+06
mean   1.000008e+03 -1.279852e-03  3.648690e-04
std    3.163730e+01  5.773752e-01  5.775576e-01
min    8.550000e+02 -9.999989e-01 -9.999999e-01
25%    9.790000e+02 -4.980895e-01 -4.954794e-01
50%    1.000000e+03  3.204826e-03  8.922588e-03
75%    1.022000e+03  5.061581e-01  5.110029e-01
max    1.180000e+03  9.999984e-01  9.999996e-01

# Make some messy data

Now this works great, and in most cases dd.read_csv or dd.read_parquet etc are the preferred way to read in large collections of data files into a dask dataframe, but real world data is often very messy and some files may be broken or badly formatted. To simulate this we are going to create some fake messy data by tweaking our example csv files. For the file `data/2000-01-05.csv` we will replace with no data and for the file `data/2000-01-07.csv` we will remove the `y` column 

In [8]:
# corrupt the data in data/2000-01-05.csv
with open('data/2000-01-05.csv', 'w') as f:
    f.write('')

In [9]:
# remove y column from data/2000-01-07.csv
import pandas as pd
df = pd.read_csv('data/2000-01-07.csv')
del df['y']
df.to_csv('data/2000-01-07.csv', index=False)

In [10]:
!head data/2000-01-05.csv

In [11]:
!head data/2000-01-07.csv

id,name,x
1042,Tim,-0.7880168340146696
972,Patricia,0.921162797786402
1002,Yvonne,0.7834199345878217
967,Alice,-0.3736619534677909
999,Jerry,0.1216683126262434
953,Charlie,-0.7133534979870275
1001,Ray,-0.2331777199594336
1049,Ray,-0.8039794111858445
960,Frank,-0.5200265922677316


# Reading the messy data

Let's try reading in the collection of files again

In [12]:
df = dd.read_csv('data/2000-*-*.csv')

In [13]:
df.head()

id      name         x         y
0  1054     Wendy -0.858609  0.475144
1   955   Michael  0.938475  0.654481
2   996     Quinn -0.681506 -0.339557
3  1039     Zelda  0.052902  0.689524
4  1009  Patricia -0.313526 -0.959217

Ok this looks like it worked, let us calculate the dataset statistics again

In [14]:
df.describe().compute()

ValueError: The columns in the computed data do not match the columns in the provided metadata
  Extra:   []
  Missing: ['y']

So what happened? 

When creating a dask dataframe from a collection of files, dd.read_csv samples the first few files in the dataset to determine the datatypes and columns available. Since it has not opened all the files it does not now if some of them are corrupt. Hence, `df.head()` works since it is only looking at the first file. `df.describe.compute()` fails because of the corrupt data in `data/2000-01-05.csv`

# Building a delayed reader

To get around this problem we are going to use a more advanced technique to build our dask dataframe. This method can also be used any time some custom logic is required when reading each file. Essentially, we are going to build a function that uses  pandas and some error checking and returns a pandas dataframe. If we find a bad data file we will either find a way to fix/clean the data or we will return and empty pandas dataframe with the same structure as the good data.

In [15]:
import numpy as np
import io

def read_data(filename):
    
    # for this to work we need to explicitly set the datatypes of our pandas dataframe 
    dtypes = {'id': int, 'name': str, 'x': float, 'y': float}
    try:
        # try reading in the data with pandas 
        df = pd.read_csv(filename, dtype=dtypes)
    except:
        # if this fails create an empty pandas dataframe with the same dtypes as the good data
        df = pd.read_csv(io.StringIO(''), names=dtypes.keys(), dtype=dtypes)
    
    # for the case with the missing column, add a column of data with NaN's
    if 'y' not in df.columns:
        df['y'] = np.NaN
        
    return df

Let's test this function on a good file and the two bad files

In [16]:
# test function on a normal file
read_data('data/2000-01-01.csv').head()

id      name         x         y
0  1054     Wendy -0.858609  0.475144
1   955   Michael  0.938475  0.654481
2   996     Quinn -0.681506 -0.339557
3  1039     Zelda  0.052902  0.689524
4  1009  Patricia -0.313526 -0.959217

In [17]:
# test function on the empty file
read_data('data/2000-01-05.csv').head()

Empty DataFrame
Columns: [id, name, x, y]
Index: []

In [18]:
# test function on the file missing the y column
read_data('data/2000-01-07.csv').head()

id      name         x   y
0  1042       Tim -0.788017 NaN
1   972  Patricia  0.921163 NaN
2  1002    Yvonne  0.783420 NaN
3   967     Alice -0.373662 NaN
4   999     Jerry  0.121668 NaN

# Assembling the dask dataframe

First we take our `read_data` function and convert it to a dask delayed function

In [19]:
from dask import delayed
read_data = delayed(read_data)

Let us look at what the function does now

In [20]:
df = read_data('data/2000-01-01.csv')
df

Delayed('read_data-1c98883d-5777-455d-96fd-d66997d89606')

It creates a delayed object, to actually run read the file we need to run `.compute()`

In [21]:
df.compute()

id      name         x         y
0      1054     Wendy -0.858609  0.475144
1       955   Michael  0.938475  0.654481
2       996     Quinn -0.681506 -0.339557
3      1039     Zelda  0.052902  0.689524
4      1009  Patricia -0.313526 -0.959217
...     ...       ...       ...       ...
86395  1008     Zelda -0.855360  0.331200
86396  1034  Patricia  0.995464  0.975729
86397   987     Frank  0.806912  0.525894
86398   947     Quinn -0.081470  0.917219
86399  1017     Alice  0.962910  0.281529

[86400 rows x 4 columns]

Now let's build a list of all the available csv files

In [22]:
# loop over all the files
from glob import glob
files = glob('data/2000-*-*.csv')
files

['data/2000-01-20.csv',
 'data/2000-01-02.csv',
 'data/2000-01-01.csv',
 'data/2000-01-17.csv',
 'data/2000-01-28.csv',
 'data/2000-01-05.csv',
 'data/2000-01-26.csv',
 'data/2000-01-21.csv',
 'data/2000-01-24.csv',
 'data/2000-01-04.csv',
 'data/2000-01-06.csv',
 'data/2000-01-13.csv',
 'data/2000-01-14.csv',
 'data/2000-01-03.csv',
 'data/2000-01-11.csv',
 'data/2000-01-27.csv',
 'data/2000-01-12.csv',
 'data/2000-01-09.csv',
 'data/2000-01-23.csv',
 'data/2000-01-16.csv',
 'data/2000-01-15.csv',
 'data/2000-01-08.csv',
 'data/2000-01-29.csv',
 'data/2000-01-19.csv',
 'data/2000-01-18.csv',
 'data/2000-01-30.csv',
 'data/2000-01-10.csv',
 'data/2000-01-25.csv',
 'data/2000-01-07.csv',
 'data/2000-01-22.csv']

Now we run the delayed read_data function on each file in the list

In [23]:
df = [read_data(file) for file in files]
df

[Delayed('read_data-9eaddd3e-3f66-4dc2-898b-5ffc0dce0bdf'),
 Delayed('read_data-ffbd26c9-f759-48b2-a943-df00f4d880a5'),
 Delayed('read_data-f3b76d32-3539-43a6-9382-0d1c60e4dafb'),
 Delayed('read_data-ee72097e-cf0a-40f6-bd15-3205e4fe7cb7'),
 Delayed('read_data-3a4f65e2-14c5-4f90-bf9a-35921d8349e1'),
 Delayed('read_data-6e864003-51fb-4294-8f1a-eec27a96bcc6'),
 Delayed('read_data-f91c0fca-d9fc-4369-bd8b-1ceed3c50e49'),
 Delayed('read_data-9008819f-e538-4f6e-adae-23f40500c708'),
 Delayed('read_data-9ae94ae7-89e5-41fa-b6de-7f22a107fadf'),
 Delayed('read_data-48fde5fd-d706-4952-98d9-8e0bb9b6c699'),
 Delayed('read_data-f9a0a62d-b373-4847-b0a7-152f4e90eadc'),
 Delayed('read_data-3f383b13-46ad-4546-aef3-90e964c622f0'),
 Delayed('read_data-31af6fc1-074b-479a-a753-658ac75abd19'),
 Delayed('read_data-7ad0b4d5-4724-44b6-9f8d-aa407e8b8a1c'),
 Delayed('read_data-a40b82d2-4a71-4216-b382-c4b3d0ea6757'),
 Delayed('read_data-012316c3-b8a3-4eba-a235-25923861f785'),
 Delayed('read_data-e9301e94-2bf6-4033-b

Then we use [dask.dataframe.from_delayed](https://docs.dask.org/en/latest/generated/dask.dataframe.from_delayed.html). This function creates a Dask DataFrame from a list of delayed objects as long as each delayed object returns a pandas dataframe. The structure of each individual dataframe returned must also be the same.

In [24]:
df = dd.from_delayed(df, meta={'id': int, 'name': str, 'x': float, 'y': float})
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
                int64  object  float64  float64
                  ...     ...      ...      ...
...               ...     ...      ...      ...
                  ...     ...      ...      ...
                  ...     ...      ...      ...
Dask Name: from-delayed, 60 tasks

Note: we provided the dtypes in the `meta` keyword to explicitly tell Dask Dataframe what kind of dataframe to expect. If we did not do this Dask would infer this from the first delayed object which could be slow if it was a large csv file

## Now let's see if this works

In [25]:
df.head()

id      name         x         y
0   983     Wendy  0.958757  0.696159
1   990   Michael  0.062365  0.389848
2   963     Wendy -0.117745  0.896765
3  1013       Dan -0.090174 -0.806392
4  1003  Patricia  0.099200  0.313263

In [26]:
df.describe().compute()

id             x             y
count  2.505600e+06  2.505600e+06  2.419200e+06
mean   1.000005e+03 -1.273076e-03  3.774278e-04
std    3.163873e+01  5.773630e-01  5.774659e-01
min    8.550000e+02 -9.999983e-01 -9.999999e-01
25%    9.790000e+02 -4.980895e-01 -4.954794e-01
50%    1.000000e+03  3.204826e-03  8.922588e-03
75%    1.022000e+03  5.061581e-01  5.110029e-01
max    1.180000e+03  9.999984e-01  9.999996e-01

## Success!

To recap, in this example, we looked at an approach to create a Dask Dataframe from a collection of many data files. Typically you would use built-in functions like `dd.read_csv` or `dd.read_parquet` to do this. Sometimes, this is not possible because of messy/corrupted files in your dataset or some custom processing that might need to be done. 

In these cases, you can build a Dask DataFrame with the following steps.

1. Create a regular python function that reads the data, performs any transformations, error checking etc and always returns a  Pandas dataframe with the same structure
2. Convert this read function to a delayed object using the `dask.delayed` function
3. Call each file in your dataset with the delayed data reader and assemble the output as a list of delayed objects
4. Used `dd.from_delayed` to covert the list of delayed objects to a Dask Dataframe 

This same technique can be used in other situations as well. Another example might be data files that require using a specialized reader, or several transformations before they can be converted to a pandas dataframe.